In [25]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import PIL
import time
import tensorflow as tf
import cv2
from matplotlib.pyplot import imshow
import shutil

In [2]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, graphPB_path): 
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()
    graph_def = None
    with tf.gfile.GFile(graph, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with self.graph.as_default():
        tf.import_graph_def(graph_def, name="")
        

    self.sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True), graph=self.graph)

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)
    
  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'unlabeled', 'ego vehicle', 'rectification border', 'out of roi', 'static', 'dynamic', 'ground',
    'road', 'sidewalk', 'parking', 'rail track', 'building', 'wall', 'fence', 'guard rail',
    'bridge', 'tunnel', 'pole', 'polegroup', 'traffic light', 'traffic sign', 'vegetation', 'terrain', 
    'sky', 'person', 'rider', 'car', 'truck', 'bus', 'caravan', 'trailer', 'train', 'motorcycle', 'bicycle', 'license plate'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [3]:
MODEL_NAME = 'xception_city_71_30000_MultEval'


_MODEL_FOLDER = {
    'xception_city_71_4500':
        'dplab_city_SingleScale_4500_Same',
    'xception_city_71_5000':
        'dplab_city_xception71_multiEval_5000_Same',
    'xception_city_71_30000_MultEval':
        'dplab_city_xception71_multiEval_30000_Same'
}

curr_dir = os.getcwd()

infer_path = os.path.join(curr_dir+'/'+_MODEL_FOLDER[MODEL_NAME]+'/export')
graph = os.listdir(infer_path)
graph = os.path.join(infer_path+'/'+graph[0])
MODEL = DeepLabModel(graph)
print('model loaded successfully!')

model loaded successfully!


In [30]:
curr_dir = os.getcwd()
DATA_PATH = os.path.join(curr_dir+'/../deeplab/datasets/kitti_seg/data_semantics/testing')
path_src = os.path.join(curr_dir+'/../deeplab/datasets/kitti_seg/data_semantics/testing/image_2')
path_dest = os.path.join(curr_dir+'/../deeplab/datasets/kitti_seg/data_semantics/testing/model_inference')
folders = ['/model_inference']
try:
    shutil.rmtree(path_dest)
    print("\n\n previous inferenced data has been deleted \n\n")
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
for folder in folders:
  try:
    os.makedirs(DATA_PATH + folder)
  except OSError as e:
    pass
all_images = sorted(os.listdir(path_src))



 previous inferenced data has been deleted 




In [33]:
curr_dir

'/raid01/qutub/projects/semantic_segmentation_models-semanseg_kitti/Model_results'

In [31]:
def seg_model(img):
    #start_time = time.clock()
    resized_im, seg_map = MODEL.run(img)
    #end_time = time.clock()
    X = seg_map
    ##Isolating SKY; 23 is labeled as sky
    X[X!=23] = 255
    X[X == 23] = 0
    #print(np.unique(X))
    RGBA_image = resized_im
    alpha = Image.fromarray(np.uint8(X))
    rgba = cv2.cvtColor(np.float32(resized_im), cv2.COLOR_RGB2RGBA)
    #print(np.unique(rgba))
    rgba[:, :, 3] = np.float32(alpha)
    
    return rgba, seg_map

In [32]:
for img in all_images:
    Img = Image.open(path_src+'/'+img)
    masked_img, seg_map = seg_model(Img)
    masked_img = Image.fromarray(np.uint8(masked_img))
    masked_img.save(path_dest+'/'+img)